<a href="https://colab.research.google.com/github/annisapraditha/tugaspraktikum_ML4/blob/main/Keg2_lagi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle==1.5.8

In [2]:
!pip install opendatasets

In [ ]:
!pip install tensorflow

In [52]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/modul 4"

In [26]:
cd /content/drive/MyDrive/modul 4/

/content/drive/MyDrive/modul 4


In [53]:
ls 

coba/        output/                                             skin-lesions/
kaggle.json  skin-lesion-analysis-toward-melanoma-detection.zip


In [ ]:
!kaggle datasets download -d wanderdust/skin-lesion-analysis-toward-melanoma-detection

In [ ]:
!unzip \*.zip && rm *.zip

In [30]:
train_dir = "train" 
test_dir = "test"
valid_dir = "val"

In [20]:
!pip install split-folders

In [ ]:
import splitfolders  

splitfolders.ratio("coba", output="output", seed=1337, ratio=(.80, .19, .01), group_prefix=None)

In [54]:
pwd

'/content/drive/My Drive/modul 4'

In [42]:
ls

coba/        output/                                             skin-lesions/
kaggle.json  skin-lesion-analysis-toward-melanoma-detection.zip


In [55]:
ls output

test/  train/  val/


In [56]:
train_path = "output/train" 
test_path  = "output/test"
val_path   = "output/val"

In [57]:
from keras.preprocessing.image import ImageDataGenerator #run

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [58]:
training_set = train_datagen.flow_from_directory(  train_path, 
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = True)

Found 2198 images belonging to 3 classes.


In [59]:
validation_set = val_datagen.flow_from_directory(    val_path, 
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 521 images belonging to 3 classes.


In [60]:
test_set = val_datagen.flow_from_directory(         test_path, 
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 31 images belonging to 3 classes.


In [61]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, Flatten, MaxPool2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

model1 = Sequential([
                     Conv2D(32, kernel_size=3, activation='relu', input_shape=(128, 128, 3)),
                     MaxPool2D(2,2),

                     Conv2D(64, kernel_size=3, activation='relu'),
                     MaxPool2D(2,2),
                     BatchNormalization(),

                     Conv2D(128, kernel_size=3, activation='relu'),
                     MaxPool2D(2,2),

                     Conv2D(256, kernel_size=3, activation='relu'),
                     MaxPool2D(2,2),

                     Dropout(0.25),

                     Flatten(),

                     Dense(128, activation='relu'),
                     Dense(3, activation='softmax')
])

model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 batch_normalization_4 (Batc  (None, 30, 30, 64)       256       
 hNormalization)                                                 
                                                      

In [62]:
model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [63]:
import time

start=time.time()
history = model1.fit(training_set,
          epochs=100,
          steps_per_epoch=10,
          validation_data=validation_set
          )

Epoch 1/100
10/10 [==============================] - 226s 24s/step - loss: 0.9035 - accuracy: 0.6219 - val_loss: 1.0098 - val_accuracy: 0.6718
Epoch 2/100
10/10 [==============================] - 200s 22s/step - loss: 0.8530 - accuracy: 0.6581 - val_loss: 1.0223 - val_accuracy: 0.6718
Epoch 3/100
10/10 [==============================] - 219s 23s/step - loss: 0.7669 - accuracy: 0.7097 - val_loss: 1.0096 - val_accuracy: 0.6718
Epoch 4/100
 8/10 [=======================>......] - ETA: 16s - loss: 0.8201 - accuracy: 0.6680

UnknownError: ignored

In [ ]:

print("waktu = ", time.time() - start, "detik")

In [ ]:
score = model1.evaluate(train_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
score = model1.evaluate(val_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
score = model1.evaluate(test_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")
plt.figure(figsize=(12,7))
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.style.use("ggplot")
plt.figure(figsize=(12,7))
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import numpy as np

pred = model1.predict(test_generator)
labels = (pred > 0.5).astype(np.int)

In [ ]:
import tensorflow as tf

model_ann = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(250,250,3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax'),
])

model_ann.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile model
model_ann.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
start=time.time()
history2 = model_ann.fit(train_generator,
          epochs=100,
          validation_data=val_generator,
          verbose=1)

In [ ]:
print("waktu = ", time.time() - start, "detik")

In [ ]:
score = model_ann.evaluate(train_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
score = model_ann.evaluate(val_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
score = model_ann.evaluate(test_generator)

print('Loss: {:.2f}'.format(score[0]))
print('Accuracy: {:.2f}'.format(score[1]))

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(12,7))
plt.plot(history2.history["loss"], label="train_loss")
plt.plot(history2.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.style.use("ggplot")
plt.figure(figsize=(12,7))
plt.plot(history2.history["accuracy"], label="train_acc")
plt.plot(history2.history["val_accuracy"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:
pred2 = model_ann.predict(test_generator)
labels = (pred2 > 0.5).astype(np.int)

In [ ]:
# model1.save("model1_diabet.h5")
# model2.save("model2_diabet.h5")

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
%load_ext tensorboard

In [ ]:
!rm -rf ./logs/

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32,64,128]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'rmsprop', 'adadelta']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import InputLayer, Activation, Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential

def train_test_model(hparams):
  model_hparams = Sequential()

  model_hparams.add(InputLayer(input_shape=[250,250,3]))

  model_hparams.add(Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(Conv2D(filters=8, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))

  model_hparams.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))

  model_hparams.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))

  model_hparams.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))

  model_hparams.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))

  model_hparams.add(Conv2D(filters=8, kernel_size=3, strides=1, padding='same', activation='relu'))
  model_hparams.add(BatchNormalization())
  model_hparams.add(MaxPool2D(pool_size=2, padding='same'))
  model_hparams.add(Dropout(hparams[HP_DROPOUT]))

  model_hparams.add(Flatten())

  # Fully Connected Layer
  model_hparams.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
  model_hparams.add(Dropout(hparams[HP_DROPOUT]))
  model_hparams.add(Dense(1, activation='softmax'))
  
  model_hparams.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
  )

  # model.fit(x_train, y_train, epochs=10) 
  model_hparams.fit(train_generator, epochs=50, validation_data=val_generator, verbose=1)
  _, accuracy = model_hparams.evaluate(x_val, y_val)
  return accuracy

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=50)

In [ ]:
['adam', 'sgd', 'adadelta', 'rmsprop', 'adamx', 'nadam']

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

In [ ]:
%tensorboard --logdir logs/hparam_tuning